Neural Network from Scratch in Python
Kevin Yang
50541650

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# load dataset
dataset = pd.read_csv('sample_dewpoint_dataset.csv')
# check all rows and column for null value
dataset.info()

In [ ]:
# define Tanh
def tanh(x):
    return np.tanh(x)

In [ ]:
cols = set(dataset.columns.values)
cols -= set(['temperature', 'humidity', 'wind_speed'])
col = set(dataset.columns.values)
col -= set(['dew_point'])
Y = dataset[list(cols)]
X = dataset[list(col)]

In [ ]:
# initialize weights randomly with mean 0 (range (-1,1))
w0 = 2*np.random.random((3,1)) -1
print(w0)

In [ ]:
# forward propagation
l0 = X
l1 = np.dot(l0, w0)
print(l1)

In [ ]:
output = tanh(l1)
print(output)

In [ ]:
# calculate the error:
output_error = Y - output
print(output_error)

In [ ]:
# backpropagation
def tanh_deriv(x):
    return 1.0 - np.tanh(x)**2

In [ ]:
# put the output error in terms of the hidden layer output
output_delta = output_error * tanh_deriv(output)
print(output_delta)

In [ ]:
err_per_weight = np.dot(l0.T, output_delta)
print(err_per_weight)

In [ ]:
w0 += err_per_weight
print("Weights nows: \n", w0)

In [ ]:
# boundle up forward propagation steps:
def forward(X, w0):
    l0 = X
    l1 = np.dot(l0, w0)
    output = tanh(l1)
    return output
output = forward(X, w0)
print(output)

In [ ]:
Y-output

In [ ]:
# bundle up the backpropagation steps:
def backprop(l0, w0, output, Y):
    output_error = Y - output
    output_delta = output_error * tanh_deriv(output)
    err_per_weight = np.dot(l0.T, output_delta)
    w0 += err_per_weight

In [ ]:
# define a function to compute the Mean Square Error (MSE)
def MSE(truth, estimate):
    return np.mean(np.square(truth - estimate))

print("MSE after scond trining step: %.3f" % MSE(Y, forward(X, w0)))

In [ ]:
def train(X, Y, w0):
    output = forward(X, w0)
    backprop(X, w0, output, Y)

In [ ]:
w0 = 2*np.random.random((3,1)) -1
for i in range(100):
    mse_before = MSE(Y, forward(X, w0))
    train(X, Y, w0)
    mse_after = MSE(Y, forward(X, w0))
    if i %10 == 0 or i == 99:
        print("i: %3i MSE before: %0.5f after: %0.5f" % (i, mse_before, mse_after))

In [ ]:
# 3-layer Network
# initial weights for this network.
w0 = 2 * np.random.random((3,5))-1
w1 = 2 * np.random.random((5,1))-1
w2 = 2 * np.random.random((1,1))-1
NN3 = [w0, w1, w2]

In [ ]:
def _forward(NN, X):
    l0 = X
    l1 = tanh(np.dot(l0, NN[0]))
    l2 = tanh(np.dot(l1, NN[1]))
    l3 = tanh(np.dot(l2, NN[2]))
    output = l3
    return (output, l2, l1, l0)
   # output = l2
    #return (output, l1, l0)
def forward(NN, X):
    return _forward(NN, X)[1]
    #return _forward(NN, X)[0]

In [ ]:
def backprop(NN, X, Y):
    output, l2, l1, l0 = _forward(NN, X)
    output_err = Y - output
    w2_delta = output_err * tanh_deriv(output)
    w2_err = np.dot(w2_delta, NN[2].T)
    w1_delta = w2_err * tanh_deriv(l2)
    w1_err = np.dot(w1_delta, NN[1].T)
    w0_delta = w1_err * tanh_deriv(l1)

    NN[0] += np.dot(X.T, w0_delta)
    NN[1] += np.dot(l1.T, w1_delta)
    NN[2] += np.dot(l2.T, w2_delta)

In [ ]:
def train(NN, X, Y):
    backprop(NN, X, Y)

In [ ]:
for i in range(100):
    mse_before = MSE(Y, forward(NN3, X))
    train(NN3, X, Y)
    mse_after = MSE(Y, forward(NN3, X))
    if i % 10 == 0 or i == 99:
        print("i: %3i MSE before: %0.5f after: %0.5f" % (i, mse_before, mse_after))

In [ ]:
# splite dataset into sets for testing and training
from sklearn.cross_validation import train_test_split
X = dataset[list(col)].values
Y = dataset[list(cols)].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size =.2, random_state=5)

In [ ]:
dataset.shape

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes = (3, 3, 3), max_iter = 500)
mlp.fit(x_train,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mlp,X,Y,cv=7)
scores

In [ ]:
from sklearn import datasets
training_stats=[]
x_fold = np.array_split(X,10)
y_fold = np.array_split(Y,10)

for k in range(10):
    x_train=list(x_fold)
    x_test = x_train.pop(k)
    x_train=np.concatenate(x_train)
    y_train=list(y_fold)
    y_test=y_train.pop(k)
    y_train=np.concatenate(y_train)
    
    mse_before = MSE(y_test, forward(NN3, x_test))
    train(NN3, x_test, y_test)
    mse_after = MSE(y_test, forward(NN3, x_test))
    print("MSE before:%0.5f after: %0.5f" % (mse_before, mse_after))
    mse_train=MSE(y_train, forward(NN3, x_train))
    print("MSE one the training set: %0.5f" % (mse_train))
    
    training_stats.append([MSE(y_test, forward(NN3, x_test)), MSE(y_train, forward(NN3, x_train))])

In [ ]:
from livelossplot import PlotLossesKeras
model = Sequential()
model.add(Dense(12, input_dim = 3, init = 'uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),callbacks=[PlotLossesKeras()],verbose=0)
print(history.history.keys())

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()